In [1]:
import numpy as np
import moviepy.editor as mp

In [2]:
clip = mp.VideoFileClip(r"D:\Speech Emotion Recognition\Speech-Emotion-Detection\S03_hin_f_admi_a_a.mov")
 
# Insert Local Audio File Path
clip.audio.write_audiofile(r"D:\Speech Emotion Recognition\Speech-Emotion-Detection\something.wav")

MoviePy - Writing audio in D:\Speech Emotion Recognition\Speech-Emotion-Detection\something.wav


MoviePy - Done.


In [ ]:
attitudelabel = filename[10:14] #label of attitude 4 letter symbol
sentencelabel = filename[15] #sentence labels could be 'a' and 'b'